# Merge all auctions into one

Combines several datasets and do some simple analyses.  


### User variables

In [1]:
import os
# setting path
os.chdir(r'..')

import drz_config
cfg = drz_config.read_config()
DATE = cfg['DATE']
VERBOSE = cfg['VERBOSE']
OPBOD = cfg['OPBOD']
SKIPSAVE = cfg['SKIPSAVE']

TAG_SINGLE = "nbconvert_instruction:remove_single_output"

if VERBOSE > 0:
    display(cfg)

{'settings_fn': '../code/assets/drz-auction-settings.ini',
 'DATE': '2022-04',
 'VERBOSE': 1,
 'OPBOD': False,
 'URL': 'http://verkoop.domeinenrz.nl/verkoop_bij_inschrijving_2022-0024',
 'EXTEND_URL': False,
 'CLOSEDDATA': True,
 'closed_data_fields': '*',
 'SKIPSAVE': False}

In [2]:
# dates of auction
# format: yyyy-mm; yyyy: year, mm: month (%Y-%m)
if OPBOD:
    AuctionDates = ['2019-11', '2019-12', 
                    '2020-01', '2020-02', '2020-03', '2021-09-09', '2021-10-10', '2021-11-11', '2021-12-12', 
                    '2022-01-01', '2022-02-02', '2022-04-04',  
                   ]
else:    
    AuctionDates = ['2014-10', '2014-11', '2014-12', # old!
                    '2015-01', '2015-02', '2015-03', # old!
                    '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', 
                    '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                    '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                    '2020-01', '2020-02', '2020-03', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                    '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', 
                    '2021-06', '2021-06-16', '2021-07-07',  '2021-07-17', '2021-08-08', '2021-08-18', '2021-09-09', '2021-09-19', '2021-10-10', '2021-10-20', '2021-11-11', '2021-11-21', '2021-12-12', '2021-12-22',
                    '2022-01-01', '2022-01-21', '2022-02-02', '2022-02-22', '2022-03-03', '2022-03-23', '2022-04-04', '2022-04-24'
                   ]  
if DATE not in ['-'.join(d.split('-')[:2]) for d in AuctionDates]:
    raise ValueError(f'{DATE} not in list that will be concatenated. Add to list.')

### Import modules 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
plt.style.use('ggplot')

### Load all data

Auctions are saved monthly in a .pkl file

In [4]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    if OPBOD:
        fn = '../data/rdw-data-{:s}-opbod.pkl'.format(
            AuctionDate
        )

    else:    
        fn = '../data/rdw-data-{:s}.pkl'.format(
            AuctionDate
        )

    if VERBOSE > 0: print(fn)
    df = pd.read_pickle(fn)
    data[AuctionDate] = df

    
    
# When there are two auctions in a single month
# HACK ALERT!!
if not OPBOD:
    data['2021-06-16'].index = data['2021-06-16'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    data['2021-06'].index = data['2021-06'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    data['2021-07-07'].index = data['2021-07-07'].index.map(lambda x: '{0}-07-{2}{1}'.format(*x.split('-')))
    data['2021-07-17'].index = data['2021-07-17'].index.map(lambda x: '{0}-07-{2}{1}'.format(*x.split('-')))
    data['2021-08-08'].index = data['2021-08-08'].index.map(lambda x: '{0}-08-{2}{1}'.format(*x.split('-')))
    data['2021-08-18'].index = data['2021-08-18'].index.map(lambda x: '{0}-08-{2}{1}'.format(*x.split('-')))
    data['2021-09-09'].index = data['2021-09-09'].index.map(lambda x: '{0}-09-{2}{1}'.format(*x.split('-')))
    data['2021-09-19'].index = data['2021-09-19'].index.map(lambda x: '{0}-09-{2}{1}'.format(*x.split('-')))
    data['2021-10-10'].index = data['2021-10-10'].index.map(lambda x: '{0}-10-{2}{1}'.format(*x.split('-')))
    data['2021-10-20'].index = data['2021-10-20'].index.map(lambda x: '{0}-10-{2}{1}'.format(*x.split('-')))
    data['2021-11-11'].index = data['2021-11-11'].index.map(lambda x: '{0}-11-{2}{1}'.format(*x.split('-')))
    data['2021-11-21'].index = data['2021-11-21'].index.map(lambda x: '{0}-11-{2}{1}'.format(*x.split('-')))
    data['2021-12-12'].index = data['2021-12-12'].index.map(lambda x: '{0}-12-{2}{1}'.format(*x.split('-')))
    data['2021-12-22'].index = data['2021-12-22'].index.map(lambda x: '{0}-12-{2}{1}'.format(*x.split('-')))
    data['2022-01-01'].index = data['2022-01-01'].index.map(lambda x: '{0}-01-{2}{1}'.format(*x.split('-')))
    data['2022-01-21'].index = data['2022-01-21'].index.map(lambda x: '{0}-01-{2}{1}'.format(*x.split('-')))
    data['2022-02-02'].index = data['2022-02-02'].index.map(lambda x: '{0}-02-{2}{1}'.format(*x.split('-')))
    data['2022-02-22'].index = data['2022-02-22'].index.map(lambda x: '{0}-02-{2}{1}'.format(*x.split('-')))
    data['2022-03-03'].index = data['2022-03-03'].index.map(lambda x: '{0}-03-{2}{1}'.format(*x.split('-')))
    data['2022-03-23'].index = data['2022-03-23'].index.map(lambda x: '{0}-03-{2}{1}'.format(*x.split('-')))
    data['2022-04-04'].index = data['2022-04-04'].index.map(lambda x: '{0}-04-{2}{1}'.format(*x.split('-')))
    data['2022-04-24'].index = data['2022-04-24'].index.map(lambda x: '{0}-04-{2}{1}'.format(*x.split('-')))


if VERBOSE > 0:
    display({'text/plain': '\nlast rows of last file'}, raw=True, metadata={"tags":(TAG_SINGLE, )})
    display(data[AuctionDates[-1]].tail(), metadata={"tags":(TAG_SINGLE, )})

load data
../data/rdw-data-2014-10.pkl
../data/rdw-data-2014-11.pkl
../data/rdw-data-2014-12.pkl
../data/rdw-data-2015-01.pkl
../data/rdw-data-2015-02.pkl
../data/rdw-data-2015-03.pkl
../data/rdw-data-2017-03.pkl
../data/rdw-data-2017-04.pkl
../data/rdw-data-2017-05.pkl
../data/rdw-data-2017-06.pkl
../data/rdw-data-2017-07.pkl
../data/rdw-data-2017-08.pkl
../data/rdw-data-2017-09.pkl
../data/rdw-data-2017-10.pkl
../data/rdw-data-2017-11.pkl
../data/rdw-data-2017-12.pkl
../data/rdw-data-2018-01.pkl
../data/rdw-data-2018-02.pkl
../data/rdw-data-2018-03.pkl
../data/rdw-data-2018-04.pkl
../data/rdw-data-2018-05.pkl
../data/rdw-data-2018-06.pkl
../data/rdw-data-2018-07.pkl
../data/rdw-data-2018-08.pkl
../data/rdw-data-2018-09.pkl
../data/rdw-data-2018-10.pkl
../data/rdw-data-2018-11.pkl
../data/rdw-data-2018-12.pkl
../data/rdw-data-2019-01.pkl
../data/rdw-data-2019-02.pkl
../data/rdw-data-2019-03.pkl
../data/rdw-data-2019-04.pkl
../data/rdw-data-2019-05.pkl
../data/rdw-data-2019-06.pkl
../d

### Consolidate format
Older files are slightly different such as naming conventions etc. Here all results are consolidated into one format.

In [5]:
# Change type of early auctions for images
if VERBOSE > 0:
    print('Fields with images look like this:\n')
    df_ = pd.DataFrame(index=data.keys(), columns=['column', 'type', 'first row'])
    for k in data:
        im_cols = data[k].columns[
            data[k].columns.str.lower().isin(['image', 'images'])
        ]
        example = data[k][im_cols[0]].values[0]
        df_.loc[k, 'type'] = str(type(example))
        df_.loc[k, 'column'] = im_cols[0]
        if isinstance(example, str):
            df_.loc[k, 'first row'] = example
        else:
            df_.loc[k, 'first row'] = len(example)
    with pd.option_context('max_row', 999):
        display(df_)


if OPBOD:
    # reformatting did not happen in IRS auctions
    pass
else:
    
    # convert string representation of list to real list
    # "['image1', 'image2']"
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        data[k]['Images'] = data[k]['Image'].apply(eval)
        data[k] = data[k].drop(columns=['Image'])

    # convert string representation of list without quotes to real list
    # "[image1, image2]"
    for k in ['2017-11',
              '2017-12',
              '2018-01',
              '2018-02',
              '2018-03',
              '2018-04']:
        data[k]['Images'] = data[k]['Images'].apply(lambda s:eval(re.sub(r"^\[(.*)\]$",r"['\1']",re.sub(' *, *',"', '",s))))
    

# Other inconsistencies
if OPBOD:
    pass
else:
    # Other column name that changed in November 2017
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        data[k] = data[k].rename(columns={
            'draw':'Draw',
            'nr':'LotNr',
            'raw':'Raw_text',
            'misc':'SupInfo',
            'footnote':'Note',
            'jfq':'jfc',
        })

Fields with images look like this:



,column,type,first row
2014-10,Images,<class 'list'>,0
2014-11,Images,<class 'list'>,0
2014-12,Images,<class 'list'>,0
2015-01,Images,<class 'list'>,0
2015-02,Images,<class 'list'>,0
2015-03,Images,<class 'list'>,0
2017-03,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14883593...
2017-04,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14906980...
2017-05,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14931056...
2017-06,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14962307...


# Merge all auctions into one data frame

In [6]:
all_data = pd.concat(data)
df = all_data.droplevel(0)

In [7]:
if VERBOSE > 0:
    # Show columns added
    col_exist = pd.DataFrame.from_dict({k:all_data.columns.isin(d.columns) for k,d in data.items()}, orient='index', columns=all_data.columns)
    col_filled = all_data.isna().all(level=0, axis=0) == False

    idx = col_exist.apply(lambda x:np.where(x)[0])

    first_occurence = col_exist.index[idx.apply(lambda x: x[0])]
    last_occurence = col_exist.index[idx.apply(lambda x: x[-1])]
    nr_occurence = idx.apply(len)
    pct_occurence = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])))

    idx = col_filled.apply(lambda x:np.where(x)[0])
    last_filled = col_filled.index[idx.apply(lambda x: x[-1] if len(x)>0 else col_exist.shape[0]-1)]
    nr_filled = idx.apply(len)
    pct_filled = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])) if len(x)>0 else 0)


    df_ = pd.DataFrame(zip(first_occurence, last_occurence, nr_occurence, last_filled, nr_filled, pct_occurence, pct_filled), 
                       index=col_exist.columns, 
                       columns=['first', 'last', 'n', 'last_filled', 'n_not_empty', 'pct_since_first', 'pct_filled'])



    for _by_rdw, df__ in df_.groupby(df_.index.str.startswith('rdw')):
        for g,df___ in df__.groupby('first'):
            if g == col_exist.index[0]:
                continue
            print(f'Fields added in {g}', end= ' ')
            if _by_rdw:
                print('by rdw api')
            else:
                print('through regex parsing')
            with pd.option_context("display.max_rows", 999):
                display(df___.sort_index().sort_values(by='last_filled', ascending=False))

Fields added in 2017-11 through regex parsing


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
used_parts,2017-11,2017-11,1,2017-11,1,1.587302,1.587302


Fields added in 2018-01 through regex parsing


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
import22_btw21,2018-01,2018-07,2,2018-07,2,3.278689,3.278689
import27_btw21,2018-01,2018-07,2,2018-07,2,3.278689,3.278689


Fields added in 2018-04 through regex parsing


...

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_verticale_belasting_koppelpunt_getrokken_voertuig,2022-02-22,2022-02-22,1,2022-02-22,1,20.0,20.0


Fields added in 2022-03-23 by rdw api


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_plaatsaanduiding_uitvoering_plaatsaanduiding_registratie_datum_5,2022-03-23,2022-03-23,1,2022-03-23,1,33.333333,33.333333
rdw_plaatsaanduiding_uitvoering_plaatsaanduiding_registratie_datum_dt_4,2022-03-23,2022-03-23,1,2022-03-23,1,33.333333,33.333333
rdw_plaatsaanduiding_uitvoering_plaatsaanduiding_registratie_datum_dt_5,2022-03-23,2022-03-23,1,2022-03-23,1,33.333333,33.333333
rdw_plaatsaanduiding_uitvoering_plaatscode_voeruig_identificatie_5,2022-03-23,2022-03-23,1,2022-03-23,1,33.333333,33.333333


Fields added in 2022-04-04 by rdw api


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_datum_eerste_tenaamstelling_in_nederland,2022-04-04,2022-04-24,2,2022-04-24,2,100.0,100.0
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_dt_142,2022-04-04,2022-04-04,1,2022-04-24,0,50.0,0.0
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_dt_148,2022-04-04,2022-04-04,1,2022-04-24,0,50.0,0.0
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_dt_147,2022-04-04,2022-04-04,1,2022-04-24,0,50.0,0.0
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_dt_146,2022-04-04,2022-04-04,1,2022-04-24,0,50.0,0.0
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_dt_145,2022-04-04,2022-04-04,1,2022-04-24,0,50.0,0.0
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_dt_144,2022-04-04,2022-04-04,1,2022-04-24,0,50.0,0.0
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_dt_143,2022-04-04,2022-04-04,1,2022-04-24,0,50.0,0.0
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_dt_141,2022-04-04,2022-04-04,1,2022-04-24,0,50.0,0.0
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_dt_133,2022-04-04,2022-04-04,1,2022-04-24,0,50.0,0.0


- - - -
# Continue with cars only

In [8]:
# select data
car=df[df.LotType == 'Personenauto'].copy()

if VERBOSE > 0:
    # print lots that are cars per auction
    df_ = pd.DataFrame(columns=['year', 'month', 'lot'])
    df_.year = pd.Series(car.index).apply(lambda x: int(x.split('-')[0]))
    df_.month = pd.Series(car.index).apply(lambda x: int(x.split('-')[1]))
    df_.lot = pd.Series(car.index).apply(lambda x: int(x.split('-')[2]))
    with pd.option_context('max_rows', 999):
        display(df_.groupby(['year', 'month']).lot.agg(['min', 'max', 'count']).sort_index())
    

display(car.tail(), metadata={"tags":(TAG_SINGLE, )})

min     max  count
year month                       
2014 10       2000    7312    136
     11       2400    8198    133
     12       2000    7306    147
2015 1        2402    8318    134
     2        2000    7318    138
     3        2400    8312    157
2017 3        2000    8318    149
     4        2000    7309    133
     5        2200    8025     75
     6        2000    7311    140
     7        2200    8178    106
     8        2001    7304    143
     9        2200    8187    133
     10       2000    7171     95
     11       2200    8305    158
     12       2000    7308    136
2018 1        2200    8170    134
     2        2000    7184    138
     3        2200    8338    186
     4        2000    7336    126
     5        2200    8226    186
     6        2000    7315    166
     7        2200    8311    132
     8        2000    7328    152
     9        2200    8183     89
     10       2000    7319     87
     11       2200    8326    127
     12       2000    7361    133
2019 1        2201    8329    135
     2        2001    7264    176
     3        2200    8318    145
     4        2000    7314    170
     5        2200    8352    158
     6        2201    9158     98
     7        2200    7254    144
     8        2200    8172     96
     9        2200    9269    156
     10       2400    7200    109
     11       2200    8213    125
     12       2200    9061     75
2020 1        2200    7205    126
     2        2200    8238    129
     3        2200    9152     65
     6        2200    7269    160
     7        2200    9271    145
     8        2200    7286    187
     9        2200    8236    111
     10       2200    7266    129
     11       2200    8260    121
     12       2201    7306    168
2021 1        2200    8123    104
     2        2200    7156    129
     3        2200    8189    148
     4        2200    7220    173
     5        2200    8158    126
     6      220006  808316    191
     7      201217  900007    152
     8      260018  812918    143
     9      220009  808919    135
     10     260110  814920    181
     11     262411  805621     97
     12     260012  807722    171
2022 1      220021  805621    115
     2      240002  810122    193
     3      220003  807723    132
     4      240004  805724     88

# Save file

In [9]:
file_name = r'../data/cars-from-all-auctions.pkl'
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    car.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    


../data/cars-from-all-auctions.pkl


# Write example results to file
based on: https://stackoverflow.com/a/33869154

In [10]:
def pandas_df_to_markdown_table(df):
    fmt = ['-----' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)


In [11]:
file_name = '../assets/example-table-of-cars.md'

example = car.tail(10).copy()

# trim some long fields
example.rdwinfo = '.. rdw info ..'
example.Raw_text = '.. raw text ..'
example.SupInfo = '.. suplm. info. ..'

# convert to md
try:
    table_text = example.to_markdown()
except ImportError:
    print('Fallback')
    table_text = pandas_df_to_markdown_table(example)

# save
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    with open(file_name,'w') as file:
        file.write(table_text)

    print('A markdown table is available as\n\t{}'.format(file_name))
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    display(example, metadata={"tags":(TAG_SINGLE, )})
    


Skip. ../assets/example-table-of-cars.md exists or saving is disabled in settings.


In [12]:
# idx = df.index.str.split('-', expand=True).droplevel(-1)
# is_last_auction = idx == idx[-1]
# subsel_reg = df.loc[is_last_auction, 'Reg'].drop_duplicates().values
# VERBOSE

In [13]:
# vc2

In [14]:
# show recurring lots (based on registration no.)

vc = df.Reg.value_counts()

# drop obvious
vc.drop(['onbekend', ''], inplace=True, errors='ignore')

# reduce output
if VERBOSE < 2:
    # only of last auction
    idx = df.index.str.split('-', expand=True).droplevel(-1)
    is_last_auction = idx == idx[-1]
    subsel_reg = df.loc[is_last_auction, 'Reg'].drop_duplicates().values
    vc = vc[vc.index.isin(subsel_reg)]
    
# show recurring
for r in vc[vc > 1].index:
    # title
    print(r, vc[r], end=': ')
    sel_reg = df.Reg==r
    print(' - '.join(list(df.loc[sel_reg, ['ItemBrand', 'ItemType', 'rdw_datum_eerste_afgifte_nederland']].drop_duplicates(ignore_index=True).iloc[0].astype(str))), end=' ')
    
    df_ = df.loc[sel_reg, list(df.columns.drop([
    'Raw_text', 'rdwinfo', 'rdw_merk_registratie_datum', 'rdw_merk_registratie_datum_dt', 'rdw_merkcode', 'Images', 'Source'
    ], errors='ignore'))].sort_values(by='Reg').T

    # convert index to nr of days since first
    ix = pd.Series(df_.columns.map(lambda x: pd.to_datetime('/'.join(x.split('-')[:2])))).diff().dt.days.copy()
    ix[0] = 0
    ix=ix.astype(int).astype(str)
    ix[0] = pd.to_datetime('/'.join(df_.columns[0].split('-')[:2])).strftime('%Y/%m')

    df_.columns=ix

    if (VERBOSE < 2) | (df_.shape[1] > 4):

        vc2 = df_.nunique(axis=1)
        print('')
        display(df_.loc[list(vc2[vc2>1].index),:])
    elif df_.shape[1] > 2:
        display(df_.loc[['Price'],:].T)
    else:
        print(*list(df_.columns))
        

44-GLG-1 3: VOLKSWAGEN - polo - 20080729.0 


,2020/07,580,59
Title,Kavel K2000079157,Kavel K2200228052,Kavel K2200248006
Price,1453.0,2222.0,1183.0
N_images,9.0,11.0,14.0
lot_counter,NaN,8052,8006
LotNr,K2000079157,K2200228052,K2200248006
LotCat,Voertuigen en onderdelen,Voertuigen en onderdelen,Voertuigen en onderdelen
NAP,,logisch,logisch
OdoKM,209.202,234.039,234.039
APKdate,17.09.2020,,
disclaim1,True,False,False


F-042-GZ 3: SYM - xs50qt - 20120302.0 


,2022/01,59,31
Title,Kavel K2200011824,Kavel K2200031817,Kavel K2200041825
Price,249.0,0.0,190.0
N_images,5.0,5.0,6.0
lot_counter,1824,1817,1825
LotNr,K2200011824,K2200031817,K2200041825
rdw_TimeStamp_x,20220107,20220311,20220408
rdw_TimeStamp_y,20220107,20220311,20220408


FK-637-V 3: KYMCO - agility 50 - 20070823.0 


...

,2022/03,31
Title,Kavel K2200231939,Kavel K2200241914
Price,0.0,363.0
N_images,6.0,7.0
lot_counter,1939,1914
LotNr,K2200231939,K2200241914
no_rdw,True,False
rdw_TimeStamp_x,20220326,20220425
rdw_TimeStamp_y,20220326,20220425


RK-916-S 2: OPEL - astra - 20171214.0 


,2022/02,59
Title,Kavel K2200027144,Kavel K2200047025
Price,4444.0,5300.0
lot_counter,7144,7025
LotNr,K2200027144,K2200047025
rdw_brandstof_emissiecode_omschrijving_1,6.0,6
rdw_TimeStamp_x,20220211,20220408
rdw_TimeStamp_y,20220211,20220408


K-533-NK 2: AUDI - rs 3 sportback - nan 


,2022/02,59
Title,Kavel K2200022605,Kavel K2200042400
Price,48000.0,48308.0
N_images,9.0,12.0
lot_counter,2605,2400
LotNr,K2200022605,K2200042400
no_inireg,True,False
rdw_TimeStamp_x,20220211,20220408
